In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open(r'\Users\gleal\.secret\yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'SA,TX'
TERM = 'Tacos'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_SA_tacos.json"
JSON_FILE

'Data/results_in_progress_SA_tacos.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [6]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_SA_tacos.json already exists. Deleting previous file...
[i] Data/results_in_progress_SA_tacos.json not found. Saving empty list to new file.
- 0 previous results found.


120

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/120 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [8]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fD626-6wEZgfGP2YT3wSIA,taquitos-west-ave-san-antonio,Taquitos West Ave,https://s3-media2.fl.yelpcdn.com/bphoto/yWNgVE...,False,https://www.yelp.com/biz/taquitos-west-ave-san...,771,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.48637, 'longitude': -98.52573}",[delivery],$,"{'address1': '2818 West Ave', 'address2': None...",+12105259888,(210) 525-9888,1415.880805
1,zulD0Ns_524wMuCAFlXxRA,tlahco-mexican-kitchen-san-antonio,Tlahco Mexican kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/PAEto8...,False,https://www.yelp.com/biz/tlahco-mexican-kitche...,779,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 29.5006724654481, 'longitude': -9...","[delivery, pickup]",$$,"{'address1': '6702 San Pedro Ave', 'address2':...",+12104550135,(210) 455-0135,1650.178441
2,pA3761zhNzc_rZXxUGIPgA,777-mini-tacos-and-birria-san-antonio,777 Mini Tacos & Birria,https://s3-media4.fl.yelpcdn.com/bphoto/wv15LW...,False,https://www.yelp.com/biz/777-mini-tacos-and-bi...,15,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 29.35898, 'longitude': -98.56145}","[delivery, pickup]",NaN,"{'address1': '3643 SW Military Dr', 'address2'...",+12107392574,(210) 739-2574,15736.282585
3,Omclc_9AiOvb2gCEQSHLjg,tacos-vitali-san-antonio-2,Tacos Vitali,https://s3-media1.fl.yelpcdn.com/bphoto/PJq-W1...,False,https://www.yelp.com/biz/tacos-vitali-san-anto...,68,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 29.54676444696915, 'longitude': -...",[],$$,"{'address1': '6433 Babcock Rd', 'address2': ''...",,,11299.333204
4,smKmeD0dcSLyYdB14eq0OQ,tacos-al-carbon-cabron-san-antonio,Tacos Al Carbon Cabron,https://s3-media3.fl.yelpcdn.com/bphoto/emL64Y...,False,https://www.yelp.com/biz/tacos-al-carbon-cabro...,16,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.0,"{'latitude': 29.500385178458828, 'longitude': ...","[delivery, pickup]",NaN,"{'address1': '6653 San Pedro Ave', 'address2':...",+12102679492,(210) 267-9492,1579.619674


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,AdEDqjy-kSAWLUJOQNcKeQ,smooth-and-savory-san-antonio,Smooth & Savory,https://s3-media3.fl.yelpcdn.com/bphoto/LlwVoz...,False,https://www.yelp.com/biz/smooth-and-savory-san...,4,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.0,"{'latitude': 29.52554, 'longitude': -98.5994}",[pickup],NaN,"{'address1': '5803 Babcock Rd', 'address2': No...",+12102672811,(210) 267-2811,9006.939336
996,xyhsrXtNQGqSaeU8ykgolQ,el-bracero-mexican-restaurant-san-antonio,El Bracero Mexican Restaurant,https://s3-media4.fl.yelpcdn.com/bphoto/mebSj0...,False,https://www.yelp.com/biz/el-bracero-mexican-re...,7,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",4.0,"{'latitude': 29.375074, 'longitude': -98.416969}",[delivery],NaN,"{'address1': '3939 E Southcross Blvd', 'addres...",+12103371686,(210) 337-1686,16296.278268
997,fkUYn1p1cYwXM25ZNoFvmw,la-esquinita-mexican-restaurant-and-groceries-...,La Esquinita Mexican Restaurant & Groceries,https://s3-media2.fl.yelpcdn.com/bphoto/WiRPmN...,False,https://www.yelp.com/biz/la-esquinita-mexican-...,24,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.0,"{'latitude': 29.278906, 'longitude': -98.719007}",[],$,"{'address1': '14624 Shepherd Rd', 'address2': ...",+12106224100,(210) 622-4100,31110.187776
998,jvlRsmRYOWuaG2fTrVRM9g,el-rodeo-de-jalisco-san-antonio-4,El Rodeo De Jalisco,https://s3-media4.fl.yelpcdn.com/bphoto/LVnevt...,False,https://www.yelp.com/biz/el-rodeo-de-jalisco-s...,21,"[{'alias': 'mexican', 'title': 'Mexican'}]",2.5,"{'latitude': 29.3977699085906, 'longitude': -9...","[pickup, delivery]",$,"{'address1': '5310 Rigsby Ave', 'address2': ''...",+12106480012,(210) 648-0012,16348.762504
999,BXpOIpz4pJbcqCOk1kuRww,sooey-shack-helotes-2,Sooey Shack,https://s3-media1.fl.yelpcdn.com/bphoto/5EiHnc...,False,https://www.yelp.com/biz/sooey-shack-helotes-2...,1,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",5.0,"{'latitude': 29.579488297215203, 'longitude': ...","[pickup, delivery]",NaN,"{'address1': '14687 Old Bandera Rd', 'address2...",+12105318918,(210) 531-8918,19636.075617


In [9]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [10]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_SA_tacos.csv.gz', compression='gzip',index=False)